The first cell collects fixture numbers which can be used as appendices to web addresses to get all the matches played in a league

In [421]:
from bs4 import BeautifulSoup
import requests
from selenium import webdriver
import time
import random
import pandas as pd
import re
import pickle

                    
#get the match page refs

def page_getter(page_address):#take page address and return all the html, includes a pause so that all scripts
                              #on page load
    driver.get(page_address)
    page = driver.execute_script("return document.documentElement.outerHTML")
    #print ("Opened   " + page_address)
    time.sleep(10 + random.random()) #pause of 15 secs plus random    
    soup = BeautifulSoup(page, "html")  #or lxml?
    return soup

def get_rows(soup):#take the html, extract all the rows of matches
    #trap bad load error - no rows to get
    if not soup.find('b', string = 'Full listing of all matches'):
        print ("Lost page")
        pageload = False
        return pageload
    #page loaded OK - crack on
    rows = soup.find('b', string = 'Full listing of all matches').find_next('tbody').find_all('tr')
    return rows


def get_fixtures (rows, lstPages):
    stop = 0
    matchcount = 0
    for row in rows[1:]: #first row is header
        
        cells = row.find_all('td')
        
        #break out for end of completed matches
        if cells[3].text == chr(160):  #if match complete, this will be a score; if space/chr(160), then match not completed
            stop = 1 #signal that no more completed fixtures #don't need this if we just return out of the func
            print (f"Reached end of completed matches. {matchcount} matches added")
            return lstPages, stop #just get out and return where we are up to
        
        fixture = row.find('p').text #grab the bit of text with fixture
        lstPages.append(fixture.replace("Fixture ID: ", "")) #reduce to just the ref number - this will become a web address later
        matchcount+= 1
    print (f"Reached end of page. {matchcount} matches added")
    return lstPages,stop    #all rows finished


In [422]:
dictkorfballpages = {'premier': ['https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60175', 
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60175%26page%3d2',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60175%26page%3d3'],
                   'southeast': ['https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60200', 
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60200%26page%3d2',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60200%26page%3d3'],
                    'norfolk1': ['https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60191',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60191%26page%3d2',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60993',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60994'],
                     'norfolk2':['https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60192',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60192%26page%3d2',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60995',
                                 'https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60996']}


In [370]:
driver = webdriver.Chrome()

soup = page_getter('https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60200%26page%3d3')
driver.quit()

<b>Full listing of all matches</b>


In [420]:
rows = get_rows(soup)
lstPages = []
def get_fixtures (rows, lstPages):
    stop = 0
    matchcount = 0
    for row in rows[1:]: #first row is header

        cells = row.find_all('td')
        
        #break out for end of completed matches
        if cells[3].text == chr(160):  #if match complete, this will be a score; if space/chr(160), then match not completed            
            print (f"Reached end of completed matches. {matchcount} matches added")
            return lstPages #just get out and return where we are up to
        
        fixture = row.find('p').text #grab the bit of text with fixture
        lstPages.append(fixture.replace("Fixture ID: ", "")) #reduce to just the ref number - this will become a web address later
        matchcount+= 1
    print (f"Reached end of page. {matchcount} matches added")
    return lstPages,stop    #all rows finished
print (get_fixtures (rows, lstPages))

Reached end of completed matches. 3 matches added
(['2236236', '2236235', '2236237'], 1)


In [400]:
place = soup.find('b', string = 'Full listing of all matches')
tbody = place.find_next('tbody')
    if x[3] != chr(160):
        print ((x)[2:8])
        lstdone.append(tr)
    lstall.append(tr)


['Harrow Vultrix Senior 1st', '22', '-', '11', '', 'Cambridge Tigers 2']
['KV 2', '18', '-', '17', '', 'Kingfisher 2']
['Norwich Knights 2', '15', '-', '15', '', 'Bearsted 1']


In [423]:
#the page list getter
driver = webdriver.Chrome()

lstPages = []

for address in dictkorfballpages['norfolk2']: #from page list above
    success = 0

    while success == 0:
        print(address)
        soup = page_getter(address) #call function and return soup
        rows = get_rows(soup) #call function and return rows with fixtures in
        if rows != False: #get_rows has said there WERE rows - scripts loaded               
            lstPages = get_fixtures(rows, lstPages)
            success = 1
            print ('Successful page')
            
print (lstPages)

driver.quit()

https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60192
Lost page
https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60192
Reached end of page. 30 matches added
Successful page
https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60192%26page%3d2
Lost page
https://w.fixtureslive.com/staticapi.aspx?a=comp_all_fixtures.ashx%3fdivisionID%3d60192%26page%3d2


AttributeError: 'tuple' object has no attribute 'append'

In [267]:
print (len(lstPages))

66


In [268]:
#get all the match pages
driver = webdriver.Chrome()

dictfcontent = {}
pageindex = []

for page_append in lstPages:
    success = 0
    while success == 0:
        match_soup = page_getter(f"https://w.fixtureslive.com/staticapi.aspx?a=statzone_match.ashx%3ffixtureID%3d{page_append}")
        fcontent = match_soup.find('div', {'class':'fcontent'})
        print (f"Try {page_append}")
        #ensure page's dynamic content was there, otherwise go again
        if fcontent is not None:
            dictfcontent[page_append] = str(fcontent) #make soup into a string so it pickles properly
            pageindex.append(page_append)
            success = 1

print(f"Finished. Got {len(dictfcontent)} fixtures")

dictfcontent ['index'] = pageindex

driver.quit()

namefile = input(f'What do you want to call this dictfcontent dictionary?')
with open(f'{namefile}.txt', 'wb') as handle:
    pickle.dump(dictfcontent, handle)
    

Try 2241244
Try 2241244
Try 2241245
Try 2241245
Try 2241246
Try 2241246
Try 2241247
Try 2241248
Try 2241248
Try 2241252
Try 2241252
Try 2241253
Try 2241253
Try 2241249
Try 2241249
Try 2241250
Try 2241250
Try 2241251
Try 2241251
Try 2246458
Try 2246458
Try 2246457
Try 2246457
Try 2246456
Try 2246454
Try 2246454
Try 2246460
Try 2246460
Try 2246461
Try 2246461
Try 2246455
Try 2246455
Try 2246462
Try 2246462
Try 2246463
Try 2246463
Try 2246464
Try 2246464
Try 2246465
Try 2246465
Try 2246466
Try 2246466
Try 2246466
Try 2246467
Try 2246467
Try 2246468
Try 2246995
Try 2246995
Try 2246470
Try 2246470
Try 2246471
Try 2246471
Try 2246471
Try 2246472
Try 2246472
Try 2246473
Try 2246473
Try 2246474
Try 2246474
Try 2246475
Try 2246475
Try 2246476
Try 2246476
Try 2246477
Try 2246477
Try 2246478
Try 2246478
Try 2246479
Try 2246479
Try 2246480
Try 2246480
Try 2246485
Try 2246485
Try 2246484
Try 2246481
Try 2246481
Try 2246481
Try 2246482
Try 2246483
Try 2246483
Try 2246488
Try 2246459
Try 2246459
Try 

The following cells all work from a fixture or match page, to extract all the information. Just provide the info in a soup - so that bit of code is still needed - a page getter (there's one above)

# Collecting data

Use the collected list of "fcontent" html chunks from match pages to extract info about each match - using a list so you can iterate through in order - collect into dictionaries and combine into one dictionary
Add the original list of matches to the dictionary as a reference list

In [269]:

def Match_info(soup):
    match_info = soup.find('div',{'class': 'flMatchDetails'})
    #print (match_info.text)
    comp = match_info.find_all('div')[0].text
    match_date = match_info.find_all('div')[1].text
    return comp, match_date

#redundant function - replaced by def Teams
#extract match squads
#I started with a for loop, and then reduced it to a list comprehension
#it is even more compressed than scorers (above)
'''def Squads(left_sections, right_sections):
    dictSquads = {}
    dictSquads['home'] = [tr.text for tr in left_sections[2].find_all('tr')]
    dictSquads['away'] = [tr.text for tr in right_sections[2].find_all('tr')]
    #remove occasional squad numbers, Reserve, and (captain)
    for pat, repl in [("\d+" , ""), ("Reserve" , ""), (r"\([^)]*\)" , "") ]:
        dictSquads ['home'] = [re.sub(pat, repl, squadname) for squadname in dictSquads['home']]
        dictSquads ['away'] = [re.sub(pat, repl, squadname) for squadname in dictSquads['away']]
    return dictSquads'''
#redundant function - replaced by def Goals
'''def Scorers(left_sections, right_sections):
    #extract scorer tags from section [1]
    home_name_tags = left_sections[1].find_all('a')
    away_name_tags = right_sections[1].find_all('a')
    dctScorers = {}
    dctScorers ['home'] = [name.text for name in home_name_tags]
    dctScorers ['away'] = [name.text for name in away_name_tags]
    #could shorten this and put straight into a dictionary
    return dctScorers'''
#redundant function - replaced by def Goals
'''def Result(left_sections, right_sections):
    home_team = left_sections[0].find('h2').text
    away_team = right_sections[0].find('h2').text
    home_score = left_sections[0].find('span', {'class': 'flHeaderScore'}).text
    away_score = right_sections[0].find('span', {'class': 'flHeaderScore'}).text
    lstResult = [home_team, away_team, home_score, away_score]
    return lstResult'''
#redundant function - code now within other functions
'''def Left_Right_sections(soup):
        #extract home and away sections
    left_sections = soup.find_all('div', {'class':'fl_left_half'})
    right_sections = soup.find_all('div', {'class': 'fl_right_half'})
    return left_sections, right_sections
'''

def Result (matchsoup):
    resultplace = matchsoup.find('h2', string = 'Match details')
    if resultplace is None:
        return ['','','','']
    home = resultplace.find_next('div', {'class':'fl_left_half'})
    home_score = home.find_next('span', {'class': 'flHeaderScore'}).text
    home_team = home.find_next('a').text
    away = resultplace.find_next('div', {'class':'fl_right_half'})
    away_score = away.find_next('span', {'class': 'flHeaderScore'}).text
    away_team = away.find_next('a').text     
    results = [home_team, away_team, home_score, away_score]
    return results

def Teams (matchsoup):
    squadplace = matchsoup.find('h3', {'class' : "flDivider"}, string = 'Teams')
    if squadplace is None:
        return {'home': [], 'away': []}
    home = squadplace.find_next('div', {'class':'fl_left_half'})
    away = squadplace.find_next('div', {'class':'fl_right_half'})
    homesquad = [tr.text for tr in home.find_all('tr')]
    awaysquad = [tr.text for tr in away.find_all('tr')]
    #remove occasional squad numbers, Reserve, and (captain)
    for pat, repl in [("\d+" , ""), ("Reserve" , ""), (r"\([^)]*\)" , "") ]:
        homesquad = [re.sub(pat, repl, squadname) for squadname in homesquad]
        awaysquad = [re.sub(pat, repl, squadname) for squadname in awaysquad]
    dictsquads = {'home': homesquad, 'away': awaysquad}
    return dictsquads

def Goals (matchsoup):
    goalsplace = matchsoup.find('h3', {'class' : "flDivider"}, string = 'Goals')
    if goalsplace is None:
        return {'home': [], 'away': []}
    home = goalsplace.find_next ('div', {'class':'fl_left_half'})
    homegoals = Goals_in_brackets ([tr.text for tr in home.find_all('tr')])
    
    away = goalsplace.find_next('div', {'class':'fl_right_half'})
    awaygoals = Goals_in_brackets ([tr.text for tr in away.find_all('tr')])
    dictscorers = {'home': homegoals, 'away': awaygoals}
    return dictscorers

def Goals_in_brackets (goalslist):
    newlist = []
    for x in goalslist:
        if re.search(r'\((.*?)\)', x): #to check there are brackets - no need to have "True" apparently
            goals = re.findall(r'\((.*?)\)',x) #to find the contents of brackets
            name = re.sub(r'\([^)]*\)', '', x) #to find brackets and remove
            for goal in range(int(goals[0])):
                newlist.append(name)
        else:
            newlist.append(x) #there weren't any brackets
    return newlist


In [321]:
souper = BeautifulSoup(dictfcontent['2255370'],'lxml')


In [363]:
def Goals (matchsoup):
    homegoals = []
    awaygoals = []
    goalsplace = matchsoup.find('h3', {'class' : "flDivider"}, string = 'Goals')
    if goalsplace is None:
        return {'home': [], 'away': []}
    home = goalsplace.find_next ('div', {'class':'fl_left_half'})
    for tr in home.find_all('tr'):
        for td in tr.find_all('td'):
            print (td, '\n')
            if td.text != 'Pen' and td.text != 'fp' and td.text != '':
                homegoals.append(td.text)
            elif re.search(r'\((.*?)\)', x):
                goals = re.findall(r'\((.*?)\)',x) #to find the contents of brackets
                name = re.sub(r'\([^)]*\)', '', x) #to find brackets and remove
                for goal in range(int(goals[0])):
                    newlist.append(name)

    away = goalsplace.find_next('div', {'class':'fl_right_half'})
    for tr in away.find_all('tr'):
        for td in tr.find_all('td'):
            if td.text != 'Pen' and td.text != 'fp' and td.text != '':
                awaygoals.append(td.text)
    dictscorers = {'home': Goals_in_brackets (homegoals), 'away': Goals_in_brackets (awaygoals)}
    return dictscorers

print (Goals(BeautifulSoup(dictfcontent['2255361'],'lxml')))

<td><img alt="Marion Catlin" class="fl_profile_pic_small" src="https://www.fixtureslive.com/uploads/uphotos/20151218145121200_profile.jpg"/></td> 



TypeError: expected string or bytes-like object

In [308]:
print (dictMatches['index'])

['2241244', '2241245', '2241246', '2241247', '2241248', '2241252', '2241253', '2241249', '2241250', '2241251', '2246458', '2246457', '2246456', '2246454', '2246460', '2246461', '2246455', '2246462', '2246463', '2246464', '2246465', '2246466', '2246467', '2246468', '2246995', '2246470', '2246471', '2246472', '2246473', '2246474', '2246475', '2246476', '2246477', '2246478', '2246479', '2246480', '2246485', '2246484', '2246481', '2246482', '2246483', '2246488', '2246459', '2246486', '2246487', '2255341', '2255342', '2255343', '2255344', '2255345', '2255346', '2255349', '2255351', '2255352', '2255350', '2255361', '2255362', '2255364', '2255363', '2255365', '2255366', '2255367', '2255368', '2255369', '2255371', '2255370']


In [292]:
#2255371, 2255370

dictMatches = {}
lstrefs = []
lstteams = []

filename = input ('What is the name of the fcontent file?')
with open(f'{filename}.txt', 'rb') as handle:
    dictfcontent = pickle.loads(handle.read())

#run functions to strip out data
for ref in dictfcontent['index']:
    
    matchsoup = BeautifulSoup(dictfcontent[ref], 'lxml')
    
    reference = ref

    competition, matchday = Match_info(matchsoup)

    dictScorers = Goals (matchsoup)

    lstResult = Result(matchsoup)

    dictSquads = Teams (matchsoup)

    lstteams.append(lstResult[0])#make a list of teams
    lstteams.append(lstResult[1])

    #add data to empty dataframe
    dfmatch = pd.DataFrame.from_dict({"date": [matchday],
             "result" : [[lstResult[x] for x in [0,2,1,3]]],
             lstResult[0] + '_squad': dictSquads['home'],
             lstResult[1] + '_squad': dictSquads['away'],
             lstResult[0] + '_scorers': dictScorers['home'],
             lstResult[1] + '_scorers': dictScorers['away']}, orient = 'index')
    dfmatch2 = dfmatch.T.reset_index()

    dictMatches[str(reference)] = dfmatch2
    lstrefs.append(reference)
    
#add a list of match reference numbers
dictMatches['index']= lstrefs
#add a list of the teams, boiled down to one of each team
lstteams = list(set(lstteams))
dictMatches['teams']= lstteams

filename = input(f'Name the dictMatches file - it is for {competition}  ')
with open(f'{filename}.txt', 'wb') as handle:
  pickle.dump(dictMatches, handle)



print (dictMatches['teams'])


What is the name of the fcontent file?Norfolk2_dictfcontent
Name the dictMatches file - it is for Norfolk Korfball League Norfolk League 2B  Norfolk2_match_facts
['Norwich Ice 3', 'Norwich City 5', 'Norwich Knights 6', 'University of East Anglia 4', 'Norwich City 4', 'Norfolk Dragons Dragons 3', 'Norwich Knights 5', 'University of East Anglia 3', 'Colchester Hornets 1', 'Norwich City 6th']


In [293]:

print (lstResult)
print ([lstResult[x] for x in [0,2,1,3]])



['University of East Anglia 3', 'Norwich Ice 3', '5', '1']
['University of East Anglia 3', '5', 'Norwich Ice 3', '1']


In [294]:
display (dictMatches['2241248'])

,index,date,result,Colchester Hornets 1_squad,University of East Anglia 3_squad,Colchester Hornets 1_scorers,University of East Anglia 3_scorers
0,0,"16:30, Sunday, 13 October, 2019","[Colchester Hornets 1, 17, University of East ...",Tom Chennell,Yolanta Connolly-Muzyczka,Tom Chennell,Dan Seagrove
1,1,None,None,Hollie Clarke,Megan Ellis,Tom Chennell,Ben Sear
2,2,None,None,Jack Craven,Freddie Elsasser,Tom Chennell,None
3,3,None,None,Katie HallMVP,Mary Emmerson,Tom Chennell,None
4,4,None,None,Alex Martin,Lydia Kittle,Tom Chennell,None
5,5,None,None,Michael Ozua,James Mackay,Jack Craven,None
6,6,None,None,Gemma Tighe,Dan Seagrove,Jack Craven,None
7,7,None,None,None,Ben Sear,Jack Craven,None
8,8,None,None,None,George Witty,Jack Craven,None
9,9,None,None,None,None,Alex Martin,None
